In [2]:
from astrapy import DataAPIClient
from astrapy.constants import VectorMetric
from astrapy.ids import UUID
# from astrapy.info import CollectionDefinition


ASTRA_DB_APPLICATION_TOKEN="AstraCS:dlNDuzLxZwsDfujcCAXlupvH:a62b0c13b526b104aadedbd1c8e73b9e6859a54054051d710cd771614bbc0038"
ASTRA_DB_API_ENDPOINT="https://9acd4f66-6b9e-4680-be3f-952a692dd9eb-us-east-2.apps.astra.datastax.com"
ASTRA_DB_KEYSPACE="defra_chatbot_keyspace"


In [5]:

# Connect and create the Database object
my_client = DataAPIClient()
my_database = my_client.get_database(
    ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)
my_database


Database(api_endpoint="https://9acd4f66-6b9e-4680-be3f-952a692dd9eb-us-east-2.apps.astra.datastax.com", token="AstraCS:dlND...", keyspace="default_keyspace")

In [7]:
my_collection = my_database.get_collection(name="hmrc_poc_discovery")
my_collection

Collection(name="hmrc_poc_discovery", keyspace="default_keyspace", database=Database(api_endpoint="https://9acd4f66-6b9e-4680-be3f-952a692dd9eb-us-east-2.apps.astra.datastax.com", token="AstraCS:dlND...", keyspace="default_keyspace"), api_options=CollectionAPIOptions(max_time_ms=None, embedding_api_key=EmbeddingAPIKeyHeaderProvider(empty)))

In [ ]:

# Populate the collection with some documents
my_collection.insert_many(
    [
        {
            "_id": UUID("018e65c9-e33d-749b-9386-e848739582f0"),
            "summary": "Riding the waves",
            "tags": ["sport"],
            "$vector": [0, 0.2, 1],
        },
        {
            "summary": "Friendly aliens in town",
            "tags": ["scifi"],
            "$vector": [-0.3, 0, 0.8],
        },
        {
            "summary": "Meeting Beethoven at the dentist",
            "$vector": [0.2, 0.6, 0],
        },
    ],
)

my_collection.update_one(
    {"tags": "sport"},
    {"$set": {"summary": "Surfers' paradise"}},
)

# Run a vector search
cursor = my_collection.find(
    {},
    sort={"$vector": [0, 0.2, 0.4]},
    limit=2,
    include_similarity=True,
)

for result in cursor:
    print(f"{result['summary']}: {result['$similarity']}")

# This would print:
#   Surfers' paradise: 0.98238194
#   Friendly aliens in town: 0.91873914

# Resource cleanup
my_collection.drop()
